In [11]:
import tensorflow as tf
import kagglehub
import os
import cv2
import numpy as np

In [2]:
!pip install tensorflow kagglehub opencv-python matplotlib numpy





#path = kagglehub.dataset_download("emmarex/plantdisease")
#print("Dataset path:", path)

train = "/kaggle/input/plant-disease-dataset/plant_disease_dataset/train"
val = "/kaggle/input/plant-disease-dataset/plant_disease_dataset/val"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Classes:", NUM_CLASSES)

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # Faster & efficient for mobile

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Lambda(preprocess_input),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("crop_disease_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved for mobile deployment")






2026-01-27 15:20:50.760563: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769527250.942624      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769527251.000333      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769527251.427219      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769527251.427263      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769527251.427266      55 computation_placer.cc:177] computation placer alr

Found 19950 files belonging to 71 classes.
Using 15960 files for training.


I0000 00:00:1769527269.904758      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Found 4275 files belonging to 71 classes.
Using 855 files for validation.
Classes: 71
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

...

I0000 00:00:1769527279.996469     135 service.cc:152] XLA service 0x7fddd4110b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769527279.996504     135 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1769527281.197447     135 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1769527286.840014     135 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


499/499 ━━━━━━━━━━━━━━━━━━━━ 80s 136ms/step - accuracy: 0.5578 - loss: 1.7196 - val_accuracy: 0.8316 - val_loss: 0.5013
Epoch 2/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 81ms/step - accuracy: 0.8717 - loss: 0.3887 - val_accuracy: 0.8667 - val_loss: 0.4034
Epoch 3/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 79ms/step - accuracy: 0.9112 - loss: 0.2739 - val_accuracy: 0.8784 - val_loss: 0.3383
Epoch 4/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 81ms/step - accuracy: 0.9344 - loss: 0.2021 - val_accuracy: 0.8772 - val_loss: 0.3389
Epoch 5/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 80ms/step - accuracy: 0.9453 - loss: 0.1565 - val_accuracy: 0.8830 - val_loss: 0.3406
Epoch 6/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 80ms/step - accuracy: 0.9593 - loss: 0.1251 - val_accuracy: 0.8795 - val_loss: 0.3656
Epoch 7/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 40s 80ms/step - accuracy: 0.9632 - loss: 0.1106 - val_accuracy: 0.8924 - val_loss: 0.3304
Epoch 8/10
499/499 ━━━━━━━━━━━━━━━━━━━━ 39s 79ms/step - accuracy: 0.9684 - loss: 0.0924 - val_accura

INFO:tensorflow:Assets written to: /tmp/tmpw0q9pcf2/assets


Saved artifact at '/tmp/tmpw0q9pcf2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 71), dtype=tf.float32, name=None)
Captures:
  140593397090256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397098704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397095056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397088720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397088336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397098512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593641714704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593641712400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397096976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593641715664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140593397

W0000 00:00:1769527723.602631      55 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1769527723.602677      55 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1769527723.752823      55 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


In [3]:
interpreter = tf.lite.Interpreter(model_path="crop_disease_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [6]:
import cv2

In [9]:
def predict_image_tflite(interpreter, image_path, class_names):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.astype(np.float32)   # ⚠️ NO /255 because preprocess_input is inside model
    img = np.expand_dims(img, axis=0)

    interpreter.set_tensor(input_details[0]["index"], img)
    interpreter.invoke()

    preds = interpreter.get_tensor(output_details[0]["index"])

    class_id = np.argmax(preds)
    confidence = np.max(preds)

    return class_names[class_id], confidence


In [18]:
print(class_names)

['Algal Leaf Spot (Jackfruit)', 'Anthracnose (Mango)', 'Aphids (Cotton)', 'Apple scab (Apple)', 'Bacterial Blight (Cotton)', 'Bacterial Canker (Mango)', 'Bacterial Leaf Spot (Pumpkin)', 'Bacterial spot (Peach)', 'Bacterial spot (Pepper, bell)', 'Bacterial spot (Tomato)', 'BacterialBlights (Sugarcane)', 'Black Rot (Cauliflower)', 'Black Spot (Jackfruit)', 'Black rot (Apple)', 'Black rot (Grape)', 'BrownSpot (Rice)', 'Cedar apple rust (Apple)', 'Cercospora leaf spot Gray leaf spot (Corn (maize))', 'Common rust (Corn (maize))', 'Cutting Weevil (Mango)', 'Die Back (Mango)', 'Downy Mildew (Pumpkin)', 'Early blight (Potato)', 'Early blight (Tomato)', 'Esca (Black Measles) (Grape)', 'Gall Midge (Mango)', 'Haunglongbing (Citrus greening) (Orange)', 'Healthy (Cauliflower)', 'Healthy (Cotton)', 'Healthy (Jackfruit)', 'Healthy (Mango)', 'Healthy (Rice)', 'Healthy (Sugarcane)', 'Healthy Leaf (Pumpkin)', 'Hispa (Rice)', 'Late blight (Potato)', 'Late blight (Tomato)', 'Leaf Mold (Tomato)', 'Leaf bli

In [17]:
label, conf = predict_image_tflite(
    interpreter,
    "/kaggle/input/plant-disease-dataset/plant_disease_dataset/test/Bacterial Blight (Cotton)/Bacterial Blight (Cotton)_1.jpg",
    class_names
)

print(label, conf)


Bacterial Blight (Cotton) 0.9997528
